## Scientific Paper Agent using LangGraph
### Overview
This project implements an intelligent research assistant that helps users navigate, understand, and analyze scientific literature using LangGraph and advanced language models. By combining various academic API with sophisticated paper processing techniques, it creates a seamless experience for researchers, students, and professionals working with academic papers.

### Motivation 
Reduce the time people invest in R&D and reading, analyzing and synthesizing academic papers. 
Key Challenes include - 
* Extensive Time commitment 
* Inefficient search processes accross fragmented database ecosystems
* Complex task synthesizingand connecting findings across multiple papers
* Stay current with new publications

## Key Components
1. State-Driven Workflow Engine
* StateGraph Architecture: Five-node system for orchestraded research.
* Decision making node: Query intent analysis and routing.
* Planning Node: Research Stategy formulation.
* Tool Execution Node: Paper Retrievel and processing.
* Judge Node: Quality validation and improvement cycles.

2. Paper Processing Integration
* Source Integration, CORE API for Comprehensive paper access.
* Document Processing, PDF content extraction, Text Structure preservation.
3. Analysis Workflow
* State-aware processing pipeline
* Multi-step validation gates
* Quality-focused improvement cycles.
* Human-in-the-loop validation options.


### Method details
#### The system requires

- Gemini API Key.
- CORE API key for paper retrieval. CORE is one of the larges online repositories for scientific papers, counting over 136 million papers, and offers a free API for personal use. A key can be requested here.
#### Technical Architecture:

- LangGraph for state orchestration.
- PDFplumber for document processing.
- Pydantic for structured data handling.

In [5]:
import io
import json
import os
import urllib3
import time

import pdfplumber
from dotenv import load_dotenv
from IPython.display import display, Markdown
from langchain_core.messages import BaseMessage, SystemMessage, ToolMessage, AIMessage
from langchain_core.tools import BaseTool, tool
from langgraph.graph import END, StateGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from typing import Annotated, ClassVar, Sequence, TypedDict, Optional
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["CORE_API_KEY"] = os.getenv('CORE_API_KEY')

## Prompts
this cell contains the prompts used in the workflow

the agent_prompt  contains a section how to use complex queries with CORE API enabling agent to solve more complex tasks.

In [ ]:
# Prompt for the initial decision making on how to reply to the user 
decision_making_prompt = """
You are an experienced scientific researcher.
Your goal is to help the use with their scientific research.

Based on the user query, decide if you need to perform a research or if you can answer the question directly.
- You should perform a research if the user query requires any supporting evidence or information.
- You should answer the question directly if the user query is a simple question that can be answered without any supporting evidence or information.
"""

# planning 
planning_prompt = """
You are an experienced scientific researcher.
Your goal is to make a new step by step plan to help the user with their scientific research.

Subtasks should not rely on any assumptions or guesses, but only rely on the information provided in the context or loop up for any additional information. 
if any feedback is provided about a previous answer, incorporate it into the new planning.
# Tools

for each substack, indicate the external tool required to complete the substack.
Tools can be one of the following:
{tools}
"""

# Prompt for the agent to answer the user query
agent_prompt = """
#IDETITY AND PURPOSE

You are an experienced scientific researcher.
Your goal is to help the user with their scientific research. You have access to a set of external tools to complete your tasks.
Follow the plan you wrote to successfully complete the task.
Add extensive inline citations to support any claim made in the answer.


# EXTERNAL KNOWLEDGE

## CORE API

The CORE API has a specific query language that allows you to explore a vast papers collection and perform complex queries. See the following table for a list of available operators:

| Operator       | Accepted symbols         | Meaning                                                                                      |
|---------------|-------------------------|----------------------------------------------------------------------------------------------|
| And           | AND, +, space          | Logical binary and.                                                                           |
| Or            | OR                     | Logical binary or.                                                                            |
| Grouping      | (...)                  | Used to prioritise and group elements of the query.                                           |
| Field lookup  | field_name:value       | Used to support lookup of specific fields.                                                    |
| Range queries | fieldName(>, <,>=, <=) | For numeric and date fields, it allows to specify a range of valid values to return.         |
| Exists queries| _exists_:fieldName     | Allows for complex queries, it returns all the items where the field specified by fieldName is not empty. |

Use this table to formulate more complex queries filtering for specific papers, for example publication date/year.
Here are the relevant fields of a paper object you can use to filter the results:
{
  "authors": [{"name": "Last Name, First Name"}],
  "documentType": "presentation" or "research" or "thesis",
  "publishedDate": "2019-08-24T14:15:22Z",
  "title": "Title of the paper",
  "yearPublished": "2019"
}

Example queries:
- "machine learning AND yearPublished:2023"
- "maritime biology AND yearPublished>=2023 AND yearPublished<=2024"
- "cancer research AND authors:Vaswani, Ashish AND authors:Bello, Irwan"
- "title:Attention is all you need"
- "mathematics AND _exists_:abstract"
"""

# Prompt for the judging step to evaluate the quality of the final answer
judge_prompt = """
You are an expert scientific researcher.
Your goal is to review the final answer you provided for a specific user query.

Look at the conversation history between you and the user. Based on it, you need to decide if the final answer is satisfactory or not.

A good final answer should:
- Directly answer the user query. For example, it does not answer a question about a different paper or area of research.
- Answer extensively the request from the user.
- Take into account any feedback given through the conversation.
- Provide inline sources to support any claim made in the answer.

In case the answer is not good enough, provide clear and concise feedback on what needs to be improved to pass the evaluation.
"""
